### Goal
Preprocessing rnaseq data

This is a tutorial with the main workflow for preparing a training dataset for a basic linear regression model. 
The output (X,Y) are all pair combination between tissues per each sample, where X is input, Y target.

In [1]:
# code autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
!pip install numpy
!pip install pandas
!pip install anndata

!pip install scanpy
!pip install argh

In [2]:
import rep.preprocessing as p
from rep.constants import ANNDATA_CST as a

%aimport

Modules to reload:
all-except-skipped

Modules to skip:



#### 1. Load raw count matrix + annotation and put it into an annodata object

In [4]:
%%writefile data.csv
T1_s1,T2_s2,T3_s1,T4_s2,T5_s1,T6_s1
G1,10,20,30,40,5,6
G2,5,10,20,30,5,6
G3,6,7,8,9,1,1

Overwriting data.csv


In [5]:
%%writefile anno_var.csv
Sample,Gender,Tissue,Individual,Technology
T1_s1,F,Tissue1,Sample1,rnaseq
T2_s2,M,Tissue2,Sample2,rnaseq
T3_s1,F,Tissue3,Sample1,rnaseq
T4_s2,M,Tissue4,Sample2,rnaseq
T5_s1,M,Tissue5,Sample1,rnaseq
T6_s1,M,Tissue6,Sample1,rnaseq

Overwriting anno_var.csv


In [11]:
%%writefile anno_obs.csv
G1,hg19,T1,chr1,1111,-
G2,hg19,T2,chr2,2222,-
G3,hg19,T3,chr3,3333,-

Writing anno_obs.csv


In [6]:
# annobj is the equivalent of the summarized experiments in R. 
# It contains 
# - annobj.X - counts
# - annobj.var - data.frame annotation object for the columns of the count matrix (in our case Tissue_Sample)
# - annobj.obs - data.frame annotation object for the rows of the count matrix (= features that we measure, in our case Genes)
# 
annobj = p.load_count_matrix("data.csv", sep=",", varanno="anno_var.csv", obsanno="anno_obs.csv")
p.print_anndata(annobj)

anndata.X
[[10. 20. 30. 40.  5.  6.]
 [ 5. 10. 20. 30.  5.  6.]
 [ 6.  7.  8.  9.  1.  1.]]
anndata.var
Index(['T1_s1', 'T2_s2', 'T3_s1', 'T4_s2', 'T5_s1', 'T6_s1'], dtype='object')
      Gender   Tissue Individual Technology
T1_s1      F  Tissue1    Sample1     rnaseq
T2_s2      M  Tissue2    Sample2     rnaseq
T3_s1      F  Tissue3    Sample1     rnaseq
T4_s2      M  Tissue4    Sample2     rnaseq
T5_s1      M  Tissue5    Sample1     rnaseq
T6_s1      M  Tissue6    Sample1     rnaseq
anndata.obs
Index(['G1', 'G2', 'G3'], dtype='object')
       1   2     3     4  5
G1  hg19  T1  chr1  1111  -
G2  hg19  T2  chr2  2222  -
G3  hg19  T3  chr3  3333  -



#### Save data to h2ad format

In [16]:
name = p.save(annobj)
name

... storing '1' as categorical
... storing '5' as categorical
... storing '1' as categorical
... storing '3' as categorical
... storing '4' as categorical


'/data/nasif12/home_if12/giurgiu/rep_gagneur/rep/notebooks/preprocessing/tmp1545316680.h5ad'

#### 2. Prepare training sample:
- select genes and tissue_samples for training
- slice the anndata object
- compute the cross tissue matrix pair (X,Y)

In [65]:
# filtering by different values in anndata.var and anndata.obs
# stored in a json format by specifying the {column over we would like to filter : array of admitted values}
# in this example we filter the anndata.var by gender (M) and anndata.obs should contain only G1 and G2
(var, obs) = p.filter_anndata_by_value(annobj, {a.VAR: {'Gender': ['M']}, a.OBS: {0: ['G1', 'G2']}})

# these are the rows and columns selected for the training set
(var,obs)

      Gender   Tissue Individual Technology
T1_s1      F  Tissue1    Sample1     rnaseq
T2_s2      M  Tissue2    Sample2     rnaseq
T3_s1      F  Tissue3    Sample1     rnaseq
T4_s2      M  Tissue4    Sample2     rnaseq
T5_s1      M  Tissue5    Sample1     rnaseq
T6_s1      M  Tissue6    Sample1     rnaseq


(['T2_s2', 'T6_s1', 'T5_s1', 'T4_s2'], ['G1', 'G2'])

In [67]:
# since we want to impute gene expression across different tissues, we
# would like to compute the cross tissue pairs per sample and not across samples

# for this purpose we can use the group_by(df, column, index_subset) 
# function to group tissues by sample

dict_samples_var = p.group_by(annobj.var, 'Individual', var)
dict_samples_var

{'Sample1': ['T6_s1', 'T5_s1'], 'Sample2': ['T2_s2', 'T4_s2']}

In [72]:
# compute cross tissue matrix pair (X, Y)
(X, Y) = p.rnaseq_cross_tissue(annobj, individuals=['Sample1','Sample2'], gene_ids=obs)

4.3 Slice anndata

anndata.X
[[20.  5. 30.  6. 10. 40.]
 [10.  5. 20.  6.  5. 30.]]
anndata.var
Index(['T2_s2', 'T5_s1', 'T3_s1', 'T6_s1', 'T1_s1', 'T4_s2'], dtype='object')
      Gender   Tissue Individual Technology
T2_s2      M  Tissue2    Sample2     rnaseq
T5_s1      M  Tissue5    Sample1     rnaseq
T3_s1      F  Tissue3    Sample1     rnaseq
T6_s1      M  Tissue6    Sample1     rnaseq
T1_s1      F  Tissue1    Sample1     rnaseq
T4_s2      M  Tissue4    Sample2     rnaseq
anndata.obs
Index(['G1', 'G2'], dtype='object')
       1   2     3     4  5
G1  hg19  T1  chr1  1111  -
G2  hg19  T2  chr2  2222  -

4.4 Build the two matrices X and Y



In [73]:
# train matrix
X

,G1,G2
T1_s1_T6_s1,10,5
T1_s1_T3_s1,10,5
T1_s1_T5_s1,10,5
T6_s1_T1_s1,6,6
T6_s1_T3_s1,6,6
T6_s1_T5_s1,6,6
T3_s1_T1_s1,30,20
T3_s1_T6_s1,30,20
T3_s1_T5_s1,30,20
T5_s1_T1_s1,5,5


In [74]:
# label matrix
Y

,G1,G2
T1_s1_T6_s1,6,6
T1_s1_T3_s1,30,20
T1_s1_T5_s1,5,5
T6_s1_T1_s1,10,5
T6_s1_T3_s1,30,20
T6_s1_T5_s1,5,5
T3_s1_T1_s1,10,5
T3_s1_T6_s1,6,6
T3_s1_T5_s1,5,5
T5_s1_T1_s1,10,5


### 3. Create toy large dataset

In [3]:
!pip install numpy
!pip install matplotlib
import random
import math
import numpy.random as nr
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

import anndata

In [4]:
# genes
genes = list(map(lambda x: "Gene_" + str(x) , list(range(1, 20001))))
genes[19990:]

['Gene_19991',
 'Gene_19992',
 'Gene_19993',
 'Gene_19994',
 'Gene_19995',
 'Gene_19996',
 'Gene_19997',
 'Gene_19998',
 'Gene_19999',
 'Gene_20000']

In [5]:
raw_counts = nr.negative_binomial(3,0.001,size=20000)
# sns.distplot(raw_counts)
raw_counts[:10]

array([2748, 7615, 1717, 2546, 2959, 3118, 1490, 5260,  630, 1818])

In [6]:
# tissues
tissues = ['Muscle', 'Blood', 'Skin', 'Lung', 'Heart', 'Stomatch', 'Colon', 'Testis', 'Brain', 'Liver']

In [7]:
# simulate how many tissues are collected per patient
distribution_tissues = nr.uniform(1,len(tissues),1000)
distribution_tissues[:10]

array([7.92409257, 1.85795368, 7.1120728 , 3.13443305, 6.07305385,
       6.03105284, 1.96623577, 8.85059353, 5.00515778, 7.85631287])

In [8]:
# generate samples - raw counts per gene per tissue
frames = []
for i in range(1,100): # patients
  
  # choose tissues
  how_many_tissues = math.floor(distribution_tissues[random.randint(1,len(distribution_tissues))])
  random.shuffle(tissues)
  t_set = tissues[:how_many_tissues]  
  
  # initialize count matrix
  counts = pd.DataFrame(columns=genes,index=map(lambda x: "Patient_" + str(i) + "_" + str(x), t_set))

  for j in range(0,len(t_set)):
    
    col_name = "Patient_" + str(i) + "_" + t_set[j]
    
    # generate raw counts per tissue per set of genes
    raw_counts = nr.negative_binomial(3,0.001,size=20000)
    counts.loc[col_name] = raw_counts.reshape(1,20000)
    
  # append expression per tissue for the patient 
  frames.append(counts)
      

df_big = pd.concat(frames)

In [9]:
# transpose the matrix to be genes x patient_tissue
count_matrix = df_big.transpose()
count_matrix.shape

(20000, 515)

In [71]:
# create annotation dataframes
# column = 0 its the index! be careful
samples_anno = pd.DataFrame(index=count_matrix.columns,columns=['Individual','Tissue'])
gender=[1,2]

for i in range(0,samples_anno.shape[0]):
  sample = samples_anno.index[i]
  [pat,number,tissue] = sample.split("_")
  
  samples_anno.at[sample, 'Individual'] = pat + "_" + str(number)
  samples_anno.at[sample, 'Tissue'] = tissue
  samples_anno.at[sample, 'Gender'] = gender[int(number)%2]

samples_anno.drop_duplicates().shape
samples_anno[:10]


,Individual,Tissue,Gender
Patient_1_Colon,Patient_1,Colon,2.0
Patient_1_Testis,Patient_1,Testis,2.0
Patient_2_Muscle,Patient_2,Muscle,1.0
Patient_2_Testis,Patient_2,Testis,1.0
Patient_3_Stomatch,Patient_3,Stomatch,2.0
Patient_4_Muscle,Patient_4,Muscle,1.0
Patient_4_Blood,Patient_4,Blood,1.0
Patient_4_Colon,Patient_4,Colon,1.0
Patient_4_Stomatch,Patient_4,Stomatch,1.0
Patient_4_Heart,Patient_4,Heart,1.0


In [72]:
genes_anno = pd.DataFrame(index=count_matrix.index)
genes_anno.shape

(100, 0)

In [73]:
# create annobj
annobj = anndata.AnnData(count_matrix)
annobj.var = samples_anno
annobj.var_names = list(samples_anno.index)
annobj.obs = genes_anno
annobj.obs_names = list(genes_anno.index)

p.print_anndata(annobj)

anndata.X ----
[[2691. 2133. 4818. ... 1695. 5303. 8355.]
 [2242. 5293. 1021. ... 2374. 5922. 1623.]
 [4557. 2922. 2317. ...  938. 5487. 1925.]
 ...
 [1168. 1726. 2143. ... 2327. 4561. 2079.]
 [1784. 2701. 1817. ...  766. 4523. 2383.]
 [2142. 1778. 4253. ... 6785. 3840.  819.]]
anndata.var ----
                       Individual    Tissue  Gender
Patient_1_Colon         Patient_1     Colon     2.0
Patient_1_Testis        Patient_1    Testis     2.0
Patient_2_Muscle        Patient_2    Muscle     1.0
Patient_2_Testis        Patient_2    Testis     1.0
Patient_3_Stomatch      Patient_3  Stomatch     2.0
Patient_4_Muscle        Patient_4    Muscle     1.0
Patient_4_Blood         Patient_4     Blood     1.0
Patient_4_Colon         Patient_4     Colon     1.0
Patient_4_Stomatch      Patient_4  Stomatch     1.0
Patient_4_Heart         Patient_4     Heart     1.0
Patient_5_Colon         Patient_5     Colon     2.0
Patient_5_Testis        Patient_5    Testis     2.0
Patient_5_Brain         Pati

In [74]:
# filter using a subset of the tissues
random.shuffle(tissues)
subset_tissue = tissues[:5]
subset_tissue

['Liver', 'Stomatch', 'Testis', 'Colon', 'Blood']

In [75]:
(train,valid,test) = p.split_by_individuals(annobj,groupby=['Gender'])
len(train),len(valid),len(test)

(599, 2)
    Individual  Gender
0    Patient_1     2.0
2    Patient_2     1.0
4    Patient_3     2.0
5    Patient_4     1.0
10   Patient_5     2.0
15   Patient_6     1.0
24   Patient_7     2.0
33   Patient_8     1.0
40   Patient_9     2.0
45  Patient_10     1.0


(360, 120, 119)

In [78]:
# slice  + compute all pairs
%time (X_train, Y_train) = p.rnaseq_cross_tissue(annobj, individuals=train,gene_ids=annobj.obs_names)
%time (X_valid, Y_valid) = p.rnaseq_cross_tissue(annobj, individuals=valid,gene_ids=annobj.obs_names)
%time (X_test, Y_test) = p.rnaseq_cross_tissue(annobj, individuals=test,gene_ids=annobj.obs_names)

Total pairs: 9578
4.4 Build the two matrices X and Y
CPU times: user 4min 44s, sys: 330 ms, total: 4min 45s
Wall time: 2min 43s
Total pairs: 3568
4.4 Build the two matrices X and Y
CPU times: user 26.4 s, sys: 3.2 s, total: 29.6 s
Wall time: 1min 13s
Total pairs: 3340
4.4 Build the two matrices X and Y
CPU times: user 19.8 s, sys: 131 ms, total: 19.9 s
Wall time: 20.9 s


In [44]:
# train data
X_train[:10]

,Gene_1,Gene_2,Gene_3,Gene_4,Gene_5,Gene_6,Gene_7,Gene_8,Gene_9,Gene_10,...,Gene_19991,Gene_19992,Gene_19993,Gene_19994,Gene_19995,Gene_19996,Gene_19997,Gene_19998,Gene_19999,Gene_20000
Patient_1_Blood_Patient_1_Liver,3762,966,974,2223,4763,1724,5866,2025,4844,5496,...,4696,1132,468,775,6098,1150,7856,2462,3628,1910
Patient_1_Liver_Patient_1_Blood,5954,2390,4451,2333,3371,3715,2096,2483,2641,7564,...,3967,1500,1003,6824,2320,7065,2660,2644,1821,1416
Patient_2_Testis_Patient_2_Brain,2766,6234,3649,1282,8075,5825,4944,108,1688,1189,...,1150,6609,1465,570,1969,4336,2644,5296,380,1993
Patient_2_Testis_Patient_2_Liver,2766,6234,3649,1282,8075,5825,4944,108,1688,1189,...,1150,6609,1465,570,1969,4336,2644,5296,380,1993
Patient_2_Brain_Patient_2_Testis,4545,3354,4068,4765,2463,4179,2773,3208,2042,2168,...,1524,6672,2991,2425,2158,4529,1961,1339,3780,2386
Patient_2_Brain_Patient_2_Liver,4545,3354,4068,4765,2463,4179,2773,3208,2042,2168,...,1524,6672,2991,2425,2158,4529,1961,1339,3780,2386
Patient_2_Liver_Patient_2_Testis,5159,1559,2982,6594,2101,3903,1786,3455,1685,1449,...,6190,2781,8495,1543,2569,2710,3536,1233,1911,1513
Patient_2_Liver_Patient_2_Brain,5159,1559,2982,6594,2101,3903,1786,3455,1685,1449,...,6190,2781,8495,1543,2569,2710,3536,1233,1911,1513
Patient_3_Heart_Patient_3_Blood,1816,2330,490,2100,4136,3471,2373,1224,2062,6767,...,2462,2730,1484,3102,3307,1028,1685,2926,345,1881
Patient_3_Heart_Patient_3_Muscle,1816,2330,490,2100,4136,3471,2373,1224,2062,6767,...,2462,2730,1484,3102,3307,1028,1685,2926,345,1881


In [47]:
Y_train[:10]

,Gene_1,Gene_2,Gene_3,Gene_4,Gene_5,Gene_6,Gene_7,Gene_8,Gene_9,Gene_10,...,Gene_19991,Gene_19992,Gene_19993,Gene_19994,Gene_19995,Gene_19996,Gene_19997,Gene_19998,Gene_19999,Gene_20000
Patient_1_Blood_Patient_1_Liver,5954,2390,4451,2333,3371,3715,2096,2483,2641,7564,...,3967,1500,1003,6824,2320,7065,2660,2644,1821,1416
Patient_1_Liver_Patient_1_Blood,3762,966,974,2223,4763,1724,5866,2025,4844,5496,...,4696,1132,468,775,6098,1150,7856,2462,3628,1910
Patient_2_Testis_Patient_2_Brain,4545,3354,4068,4765,2463,4179,2773,3208,2042,2168,...,1524,6672,2991,2425,2158,4529,1961,1339,3780,2386
Patient_2_Testis_Patient_2_Liver,5159,1559,2982,6594,2101,3903,1786,3455,1685,1449,...,6190,2781,8495,1543,2569,2710,3536,1233,1911,1513
Patient_2_Brain_Patient_2_Testis,2766,6234,3649,1282,8075,5825,4944,108,1688,1189,...,1150,6609,1465,570,1969,4336,2644,5296,380,1993
Patient_2_Brain_Patient_2_Liver,5159,1559,2982,6594,2101,3903,1786,3455,1685,1449,...,6190,2781,8495,1543,2569,2710,3536,1233,1911,1513
Patient_2_Liver_Patient_2_Testis,2766,6234,3649,1282,8075,5825,4944,108,1688,1189,...,1150,6609,1465,570,1969,4336,2644,5296,380,1993
Patient_2_Liver_Patient_2_Brain,4545,3354,4068,4765,2463,4179,2773,3208,2042,2168,...,1524,6672,2991,2425,2158,4529,1961,1339,3780,2386
